In [6]:
!pip install langchainhub

In [4]:
from langchain.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain import hub

import os
from dotenv import load_dotenv

In [5]:
# Cargar variables de entorno
load_dotenv()

# Configurar el motor de OpenAI
model = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 17


In [2]:
# Inicializamos db
db = SQLDatabase.from_uri("sqlite:///chinook.db")

In [7]:
# Pull down prompt
prompt = hub.pull("rlm/text-to-sql")

In [8]:
# Crear chain con LangChain Expression Language
inputs = {
    "table_info": lambda x: db.get_table_info(),
    "input": lambda x: x["question"],
    "few_shot_examples": lambda x: "",
    "dialect": lambda x: db.dialect,
}
sql_response = (
    inputs
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

# Call with a given question
sql_response.invoke({"question": "Enumera las ventas totales por país. ¿Los clientes de qué país gastaron más?"})

'SQLQuery: "SELECT country, SUM(total) as total_sales FROM invoices JOIN customers ON invoices.customer_id = customers.customer_id GROUP BY country ORDER BY total_sales DESC LIMIT 1"'

In [9]:
# Call with a given question
sql_response.invoke({"question": "¿De qué país hay más artistas?"})

'SQLQuery: SELECT pais, COUNT(*) AS num_artistas FROM artistas GROUP BY pais ORDER BY num_artistas DESC LIMIT 1;'